In [45]:
from pxr import Usd,UsdGeom,Sdf

In [2]:
stage = Usd.Stage.Open("D:/Kitchen_set/Kitchen_set_instanced.usd")

In [44]:
for prim in stage.Traverse():
    if prim.IsInstance():
        # instanceable なPrimを列挙する
        # Primから、InstanceのPrototype（ベース）のPrimを取得する
        protoPrim = prim.GetPrototype()

In [4]:
# instanceable が指定されたPrimは、
# StageにPrototypePrimとして登録される。
prototypes = stage.GetPrototypes()
print(prototypes[0].IsPrototype())

True


In [7]:
canA = stage.GetPrimAtPath('/Kitchen_set/Props_grp/West_grp/TinCanA_1')
canB = stage.GetPrimAtPath('/Kitchen_set/Props_grp/West_grp/TinCanA_2')
# 同じモデルなので、Prototypeは共通
print(canA.GetPrototype())
print(canB.GetPrototype())

Usd.Prim(</__Prototype_2>)
Usd.Prim(</__Prototype_2>)


In [46]:
stage = Usd.Stage.CreateInMemory()

primA = stage.DefinePrim("/sampleA")
primA.GetReferences().AddReference('./cube.usd')
primA.SetInstanceable(True)
primB = stage.DefinePrim("/sampleB")
primB.SetInstanceable(True)
primB.GetReferences().AddReference('./cube.usd')

primC = stage.DefinePrim("/sampleC")
primC.GetReferences().AddReference('./cube.usd')

# Instanceの場合はChildはみつからない
print(primB.GetChildren())
# 普通なら見つかる
print(primC.GetChildren())

[]
[Usd.Prim(</sampleC/cube>)]


In [47]:
stage.GetRootLayer().Export("./instance.usd")

True

In [38]:
for i in primB.GetPrototype().GetChildren():
    print(i)

Usd.Prim(</__Prototype_1/cube>)


In [49]:
prototypes = stage.GetPrototypes()
prototype = prototypes[0]

In [50]:
prototype.GetInstances()

[Usd.Prim(</sampleA>), Usd.Prim(</sampleB>)]

In [61]:
# InstanceのPrimも GetPrimAtPathで取得が可能
proxy = stage.GetPrimAtPath("/sampleA/cube")
# Proxyの場合、値の追加や変更を使用するとエラーになる
proxy.CreateAttribute('sample',Sdf.ValueTypeNames.String).Set('hoge')

ErrorException: 
	Error in 'pxrInternal_v0_21__pxrReserved__::UsdStage::_ValidateEditPrim' at line 1297 in file D:\work\GithubRepo\USD\pxr\usd\usd\stage.cpp : 'Cannot create property spec at path </sampleA/cube>; authoring to an instance proxy is not allowed.'

In [66]:
# ProxyからPrototype側でのPathを取得
protoPrim = proxy.GetPrimInPrototype()
protoPrim.CreateAttribute('sample',Sdf.ValueTypeNames.String).Set('hoge')


ErrorException: 
	Error in 'pxrInternal_v0_21__pxrReserved__::UsdStage::_ValidateEditPrim' at line 1290 in file D:\work\GithubRepo\USD\pxr\usd\usd\stage.cpp : 'Cannot create property spec at path </__Prototype_1/cube>; authoring to an instancing prototype is not allowed.'